In [62]:
import requests
from bs4 import BeautifulSoup as bs
import json

GET_INN_URL = 'https://yandex.ru/search/?text='
EGRUL_URL = 'https://egrul.itsoft.ru/'
REPUTATION_URL = 'https://vbankcenter.ru/contragent/'


def get_company_list_by_product_metalloprokat(product_name):
    resut = json.dumps({})

    TARGET_URL = 'https://www.metalloprokat.ru/company/?q='
    
    site_raw = requests.get(TARGET_URL+product_name).text
    parsed = bs(site_raw, features='lxml')
    comp_count = int(parsed.find_all('div', {'id': 'tabs'})[
                 0].find_all('span', {'class': 'count'})[1].text)

    if comp_count < 1:
        return resut
    
    companies = parsed.find_all('div', {'class': 'title'})

    sellers = dict()

    for company in companies:
        try:
            seller_name = company.find_all('a')[0].text[1:-1]
            sellers[seller_name] = []
            sellers[seller_name].append(company.find_all('a')[0]['href'])
        except:
            pass

    reviews = parsed.find_all('li', {'class': 'links_comment item medium float-left clearfix'})
    for review in reviews:
        try:
            print(review.a['data-href'])
        except:
            pass


    for seller in sellers:
        try:
            raw_for_inn = requests.get(GET_INN_URL+seller+'+ИНН').text
            parsed = bs(raw_for_inn, features='lxml')
            inn = parsed.find_all('div', {'class': 'serp-list serp-list_right_yes serp-list_complementary_yes'})[
                0].find_all('div', {'class': 'KeyValue-ItemValue'})[0].text
            sellers[seller].append(inn)
            sellers[seller].append(json.loads(
                requests.get(EGRUL_URL+inn+'.json').text))
        except:
            pass

    resut = json.dumps(sellers)
    
    return resut



def get_company_list_by_product(product_name):

    json_res = get_company_list_by_product_metalloprokat(product_name)
    with open(product_name+'.txt', 'w') as rfile:
        rfile.write(json_res)
    pass



In [63]:
req_str = input('Enter request:')
get_company_list_by_product(req_str)

https://bcm-metall.metalloprokat.ru/reviews
https://st-kom.metalloprokat.ru/reviews
http://schekinoazot.metalloprokat.ru/reviews


***Проверка методики разложения наименования на группы***

In [64]:
# Загрузка тестовых данных
import pandas as pd

input_data = pd.read_excel('task3/poiskpostav_v1.xlsx')

In [65]:
# Подготовка шаблонов для разбора строк
import re
# Шаблон для ГОСТов
patGOST = re.compile(r'(ГОСТ\s?Р?\s?(?:ИСО)?\s?\d*[\/]?\d*)')
# Шаблон для метрических параметров
patMetricSize = re.compile(r'[мМ]\d{1,}[хХ]?\d{1,}')

In [66]:
input_data['ГОСТ'] = None
for i in range(len(input_data)):
    gosts = re.finditer(patGOST, input_data['Наименование'][i])
    for gost in gosts:
        if gost.lastindex>0:
            prev = ''
            
            if input_data['ГОСТ'][i]:
                prev = input_data['ГОСТ'].loc[i]+', '
            input_data['ГОСТ'][i] = prev + gost[0]
           

input_data

/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_67062/2703321491.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['ГОСТ'][i] = prev + gost[0]


,№,Наименование,ГОСТ
0,1,* Манжета М50х70 ГОСТ 22704,ГОСТ 22704
1,2,* Манжета М65х90 ГОСТ 22704,ГОСТ 22704
2,3,* Манжета М220х250 ГОСТ 22704,ГОСТ 22704
3,4,* Манжета М60х80 ГОСТ 22704,ГОСТ 22704
4,5,* Манжета резиновая армированная для валов 1.2...,ГОСТ 8752
5,6,* Пластина техническая 2Н-I-ТМКЩ-С-4 ГОСТ 7338,ГОСТ 7338
6,7,"* Рукав всасывающий В-1-75-У рабочий вакуум 0,...",ГОСТ 5398
7,8,* Рукав с текстильным каркасом В(II)-16-25-38-...,ГОСТ 18698
8,9,* Пластина техническая 1Н-I-ТМКЩ-С-5 ГОСТ 7338,ГОСТ 7338
9,10,* Манжета резиновая армированная для валов 1.2...,ГОСТ 8752


In [102]:
# Парсинг Яндекса по номенклатуре

def getTopFromYandex(request_str):
    # Получаем наменклатурную позицию и возвращаем лист пар URL описание
    result_url = []
    result_description = []
    result = []
    SEARCH_URL = "https://yandex.ru/search/direct?text="
    SEARCH_URL_POSTFIX = "&filters_docs=direct_cm"
    request_str = "купить "+request_str[1:]
    
    raw_result = requests.get(SEARCH_URL+request_str+SEARCH_URL_POSTFIX).text

    parsed_result = bs(raw_result, features='lxml')
    
    cards = parsed_result.find_all('li', {'class':"serp-item desktop-card"})

    for card in cards:
        links = card.find_all('a', {'target':"_blank"})
        for link in links:
            result_url.append(link.get('href'))
        descriptions = card.find_all('span',{'role':'text'})
        
        for description in descriptions:
            result_description.append(description.text)

    for _ in range(min(len(result_url), len(result_description))):
        result.append([result_url[_], result_description[_]])

    return result

In [104]:
# Пример парсинга имеющейся номенклатуры 

res = []
for i in range(len(input_data)):
    res.append(getTopFromYandex(input_data['Наименование'][i]))

res_json = json.dumps(res)
with open('curdump.txt','w') as res_file:
    res_file.write(res_json)
    